# Find proteins outside the 8p loss event (trans) that are commonly different between samples with and without the event

# Only looking at interacting proteins

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_filtered.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,brca_Database_ID,lscc_Database_ID,luad_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
AAR2,NP_001258803.1,NP_001258803.1,NP_001258803.1,NP_001258803,0.888631,0.325501,0.972062,0.365710,0.946253,0.950328,0.138113,0.119317,0.008190,0.174323,0.024097,0.063430
ABCE1,NP_001035809.1,NP_001035809.1,NP_001035809.1,NP_001035809,0.924653,0.987370,0.584665,0.059979,0.994144,0.821133,0.068978,0.003831,0.068507,0.311653,-0.001090,-0.064198
ABCF3,NP_060828.2,NP_060828.2|NP_001338227.1|NP_001338229.1|NP_0...,NP_060828.2|NP_001338227.1|NP_001338229.1|NP_0...,NP_060828,0.924653,0.982661,0.282020,0.002957,0.980903,0.979021,0.107994,0.003609,0.139489,0.754682,-0.010976,0.010350
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001290204,0.835463,0.250509,0.291028,0.266241,0.495917,0.952219,0.274622,0.133618,0.168614,0.254131,-0.322379,-0.048535
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001087,0.835463,0.250509,0.291028,0.266241,0.495917,0.952219,0.274622,0.133618,0.168614,0.254131,-0.322379,-0.049849
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204,0.835463,0.250509,0.291028,0.266241,0.666328,0.952219,0.274622,0.133618,0.168614,0.254131,0.180992,-0.048535
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001087,0.835463,0.250509,0.291028,0.266241,0.666328,0.952219,0.274622,0.133618,0.168614,0.254131,0.180992,-0.049849
ACLY,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001290204.1|NP_942127.1,NP_001290204,0.835463,0.250509,0.291028,0.853804,0.495917,0.952219,0.274622,0.133618,0.168614,-0.083106,-0.322379,-0.048535
ACLY,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001290204.1|NP_942127.1,NP_001087,0.835463,0.250509,0.291028,0.853804,0.495917,0.952219,0.274622,0.133618,0.168614,-0.083106,-0.322379,-0.049849


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,AAR2,NP_001258803.1,0.888631,0.138113
1,brca,ABCE1,NP_001035809.1,0.924653,0.068978
2,brca,ABCF3,NP_060828.2,0.924653,0.107994
3,brca,ACTB,NP_001092.1,0.931761,0.054662
4,brca,AGAP1,NP_001032208.1|NP_055729.2,0.924653,-0.072918
5,brca,AGPS,NP_003650.1,0.924653,-0.213403
6,brca,AHCYL1,NP_006612.2|NP_001229602.1,0.945538,0.037484
7,brca,ALG2,NP_149078.1,0.924653,0.062978
8,brca,AMOTL2,NP_001265612.1|NP_057285.3|NP_001265614.1,0.924653,-0.134768
9,brca,AP2B1,NP_001025177.1|NP_001273.1,0.943485,0.025034


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [7]:
prots.groupby("cancer_type").count()[["protein"]]

,protein
cancer_type,
brca,3
colon,8
lscc,43
luad,14


## Find how many cancers each protein was different in

In [8]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
ATP6V1A,"[brca, colon, luad]",3
ATP6V1E1,"[colon, lscc, luad]",3
ATP6V1H,"[colon, lscc, luad]",3
ATP6V1D,"[brca, luad]",2
ATP6V1F,"[colon, luad]",2
ALG2,"[lscc, luad]",2
PPP2R1A,[brca],1
HSPA4L,[colon],1
NARS,[colon],1


In [9]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

3     3
2     3
1    53
Name: num_cancers, dtype: int64

## Save results

In [10]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}_filtered.tsv"
prots_summary.to_csv(output_file, sep="\t")